In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
pd.options.display.max_columns=100
pd.options.display.max_rows=100
from encoding import FreqeuncyEncoding
from custom_estimator_regressor import Estimator
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.externals import joblib
import datetime

### add more historical vars at a member_id level
### think more about neighbouring vars

In [2]:
train=pd.read_csv('train_1.csv')
test=pd.read_csv('test_1.csv')
target=train.amount_spent_per_room_night_scaled

In [3]:
train.drop('amount_spent_per_room_night_scaled',axis=1,inplace=True)
print train.shape,test.shape
df=pd.concat((train,test),axis=0)
print df.shape

(341424, 42) (146765, 42)
(488189, 42)


In [4]:
grp_ids=train.memberid

In [5]:
from sklearn.model_selection import GroupKFold
kf=GroupKFold(n_splits=5)
folds=[(i[0].tolist(),i[1].tolist()) for i in kf.split(train,target,grp_ids)]

In [6]:
df.fillna(-1,inplace=True)

In [7]:
df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,checkin_out_delta,booking_checkin_delta,booking_checkout_delta,booking_mon,booking_day,booking_year,booking_dow,booking_week_no,booking_day_of_year,booking_quarter,booking_week_of_yr,checkin_mon,checkin_day,checkin_year,checkin_dow,checkin_week_no,checkin_day_of_year,checkin_quarter,checkin_week_of_yr
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2018-04-05,2018-04-05,2018-04-06,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,1,0,1,4,5,2018,3,14,95,2,14,4,5,2018,3,14,95,2,14
1,03930f033646d073462b35d411616323597715ac4fc398...,2015-01-23,2015-04-11,2015-04-16,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,5,78,83,1,23,2015,4,4,23,1,4,4,11,2015,5,15,101,2,15
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,2015-01-28,2015-02-01,2015-02-05,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,4,4,8,1,28,2015,2,5,28,1,5,2,1,2015,6,5,32,1,5
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,2015-05-02,2015-06-11,2015-06-16,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5,40,45,5,2,2015,5,18,122,2,18,6,11,2015,3,24,162,2,24
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,2015-09-02,2015-12-14,2015-12-19,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5,103,108,9,2,2015,2,36,245,3,36,12,14,2015,0,51,348,4,51


In [8]:
df['child_adult_ratio']= df.numberofchildren/ (1+df.numberofadults)
df['num_single_rm_nts_adults']= df.roomnights* df.numberofadults
df['num_single_rm_nts_children']= df.roomnights* df.numberofchildren
df['adult_Frac']= df.numberofadults/ (df.total_pax)
df['child_Frac']= df.numberofchildren/ df.total_pax
df['pax_rm_nts']=df.total_pax * df.roomnights
df['is_children']= (df.numberofchildren>0).astype('int')
df['cont1']=df.booking_checkin_delta/df.booking_checkout_delta
df['delta_rmnts_checin_in_out_delta']= df['roomnights']- df['checkin_out_delta']
df['calc_pax_rm_nts']=df.total_pax * df.checkin_out_delta

In [9]:
df.shape

(488189, 52)

In [10]:
df.booking_date=pd.to_datetime(df.booking_date)
df.checkin_date=pd.to_datetime(df.checkin_date)
df.checkout_date=pd.to_datetime(df.checkout_date)

In [11]:
def res_hist_vars(res_id,df1):
    temp_df=df1[df1.resort_id==res_id]
    temp_df=temp_df.sort_values('booking_date')
    temp_df['visit_number_resort']=temp_df.booking_date.rank()
    
    temp_df['last_total_pax_diff_resort']=(temp_df['total_pax']-temp_df['total_pax'].shift(1))
    
    temp_df['last_roomnights_diff_resort']=(temp_df['roomnights']-temp_df['roomnights'].shift(1))
    
    temp_df['last_adult_diff_resort']=(temp_df['numberofadults']-temp_df['numberofadults'].shift(1))
    
    temp_df['pax_cum_mean_resort']=temp_df.total_pax.shift(1).expanding().mean()
    temp_df['numberofadults_cum_mean_resort']=temp_df.numberofadults.shift(1).expanding().mean()
    temp_df['roomnights_cum_mean_resort']=temp_df.roomnights.shift(1).expanding().mean()
    temp_df['checkin_out_delta_cum_mean_resort']=temp_df.checkin_out_delta.shift(1).expanding().mean()
    temp_df['booking_checkin_delta_cum_mean_resort']=temp_df.booking_checkin_delta.shift(1).expanding().mean()
    temp_df['booking_checkout_delta_cum_mean_resort']=temp_df.booking_checkout_delta.shift(1).expanding().mean()
    
    return temp_df

def state_resort_hist_vars(state_id_res,df1):
    temp_df=df1[df1.state_code_resort==state_id_res]
    temp_df=temp_df.sort_values('booking_date')
    temp_df['visit_number_state_res']=temp_df.booking_date.rank()

    temp_df['last_total_pax_diff_state_res']=(temp_df['total_pax']-temp_df['total_pax'].shift(1))

    temp_df['last_roomnights_diff_state_res']=(temp_df['roomnights']-temp_df['roomnights'].shift(1))
    
    temp_df['last_adult_diff_state_res']=(temp_df['numberofadults']-temp_df['numberofadults'].shift(1))
    
    temp_df['pax_cum_mean_state_res']=temp_df.total_pax.shift(1).expanding().mean()
    
    temp_df['numberofadults_cum_mean_state_res']=temp_df.numberofadults.shift(1).expanding().mean()
    temp_df['roomnights_cum_mean_state_res']=temp_df.roomnights.shift(1).expanding().mean()
    temp_df['checkin_out_delta_cum_mean_state_res']=temp_df.checkin_out_delta.shift(1).expanding().mean()
    temp_df['booking_checkin_delta_cum_mean_state_res']=temp_df.booking_checkin_delta.shift(1).expanding().mean()
    temp_df['booking_checkout_delta_cum_mean_state_res']=temp_df.booking_checkout_delta.shift(1).expanding().mean()
    
    
    return temp_df

def state_ind_hist_vars(state_id_ind,df1):
    temp_df=df1[df1.state_code_residence==state_id_ind]
    temp_df=temp_df.sort_values('booking_date')
    temp_df['visit_number_state_ind']=temp_df.booking_date.rank()
    temp_df['last_total_pax_diff_state_ind']=(temp_df['total_pax']-temp_df['total_pax'].shift(1))
    
    temp_df['last_roomnights_diff_state_ind']=(temp_df['roomnights']-temp_df['roomnights'].shift(1))
    
    temp_df['last_adult_diff_state_ind']=(temp_df['numberofadults']-temp_df['numberofadults'].shift(1))
    
    temp_df['pax_cum_mean_state_ind']=temp_df.total_pax.shift(1).expanding().mean()
    temp_df['numberofadults_cum_mean_state_ind']=temp_df.numberofadults.shift(1).expanding().mean()
    temp_df['roomnights_cum_mean_state_ind']=temp_df.roomnights.shift(1).expanding().mean()
    temp_df['checkin_out_delta_cum_mean_state_ind']=temp_df.checkin_out_delta.shift(1).expanding().mean()
    temp_df['booking_checkin_delta_cum_mean_state_ind']=temp_df.booking_checkin_delta.shift(1).expanding().mean()
    temp_df['booking_checkout_delta_cum_mean_state_ind']=temp_df.booking_checkout_delta.shift(1).expanding().mean()
    
    
    return temp_df

In [12]:
def mem_hist_vars(mem):
    temp_df=df[df.memberid==mem]
    temp_df=temp_df.sort_values('booking_date')
    temp_df['visit_number']=temp_df.booking_date.rank()
    temp_df['last_trip_diff']=(temp_df['booking_date']-temp_df['booking_date'].shift(1)).dt.days
    temp_df['next_trip_diff']=(temp_df['booking_date']-temp_df['booking_date'].shift(-1)).dt.days
    
    temp_df['last_total_pax_diff']=(temp_df['total_pax']-temp_df['total_pax'].shift(1))
    
    temp_df['last_roomnights_diff']=(temp_df['roomnights']-temp_df['roomnights'].shift(1))
    
    temp_df['last_adult_diff']=(temp_df['numberofadults']-temp_df['numberofadults'].shift(1))
    
    temp_df['last_trip_diff_check_in']=(temp_df['checkin_date']-temp_df['checkin_date'].shift(1)).dt.days
    temp_df['last_trip_diff_check_out']=(temp_df['checkout_date']-temp_df['checkout_date'].shift(1)).dt.days
    
    temp_df['next_trip_diff_check_in']=(temp_df['checkin_date']-temp_df['checkin_date'].shift(-1)).dt.days
    temp_df['next_trip_diff_check_out']=(temp_df['checkout_date']-temp_df['checkout_date'].shift(-1)).dt.days
    
    temp_df['last_cacl_rmnts_diff_check_in']=(temp_df['checkin_out_delta']-temp_df['checkin_out_delta'].shift(1))
    
    temp_df['pax_cum_mean']=temp_df.total_pax.shift(1).expanding().mean()
    
    temp_df['numberofadults_cum_mean']=temp_df.numberofadults.shift(1).expanding().mean()
    temp_df['roomnights_cum_mean']=temp_df.roomnights.shift(1).expanding().mean()
    temp_df['checkin_out_delta_cum_mean']=temp_df.checkin_out_delta.shift(1).expanding().mean()
    
    temp_df['booking_checkin_delta_cum_mean']=temp_df.booking_checkin_delta.shift(1).expanding().mean()
    temp_df['booking_checkout_delta_cum_mean']=temp_df.booking_checkout_delta.shift(1).expanding().mean()
    return temp_df


In [13]:
mem_list=df.memberid.unique().tolist()

In [14]:
%%time
from multiprocessing import Pool,cpu_count
try:
    pool = Pool(cpu_count())
    data_outputs = pool.map(mem_hist_vars, mem_list)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

CPU times: user 8min 30s, sys: 21.1 s, total: 8min 51s
Wall time: 10min 28s


In [15]:
%%time
temp_df=pd.DataFrame(np.row_stack(data_outputs),columns=data_outputs[0].columns)

CPU times: user 2min 56s, sys: 1.45 s, total: 2min 57s
Wall time: 2min 57s


In [16]:
temp_df.shape

(488189, 69)

In [17]:
%%time
resort_list=df.resort_id.unique().tolist()
temp=pd.DataFrame()
for i,res in enumerate(resort_list):
    temp=pd.concat((temp,res_hist_vars(res,temp_df)),axis=0)
temp_df=temp    
print temp_df.shape

(488189, 79)
CPU times: user 4min 34s, sys: 2min 9s, total: 6min 43s
Wall time: 3min 52s


In [18]:
%%time
state_list=df.state_code_residence.unique().tolist()
temp=pd.DataFrame()
for i,state in enumerate(state_list):
    temp=pd.concat((temp,state_ind_hist_vars(state,temp_df)),axis=0)
temp_df=temp    

CPU times: user 7min 48s, sys: 4min 45s, total: 12min 33s
Wall time: 5min 59s


In [19]:
temp_df.shape

(488189, 89)

In [20]:
%%time
state_list=df.state_code_resort.unique().tolist()
temp=pd.DataFrame()
for i,state in enumerate(state_list):
    temp=pd.concat((temp,state_resort_hist_vars(state,temp_df)),axis=0)
temp_df=temp    

CPU times: user 2min 11s, sys: 1min 18s, total: 3min 29s
Wall time: 1min 39s


In [21]:
temp_df.shape

(488189, 99)

In [22]:
temp_df=pd.DataFrame(df['reservation_id']).merge(temp_df)

In [23]:
temp_df.shape

(488189, 99)

In [24]:
temp_df.booking_date=temp_df.booking_date.dt.date.astype('str')
temp_df.checkin_date=temp_df.checkin_date.dt.date.astype('str')
temp_df.checkout_date=temp_df.checkout_date.dt.date.astype('str')

In [25]:
df=temp_df

In [26]:
df.shape

(488189, 99)

In [27]:
df.fillna(-1,inplace=True)

In [28]:
df.shape

(488189, 99)

In [29]:
df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,checkin_out_delta,booking_checkin_delta,booking_checkout_delta,booking_mon,booking_day,booking_year,booking_dow,booking_week_no,booking_day_of_year,booking_quarter,booking_week_of_yr,checkin_mon,checkin_day,checkin_year,checkin_dow,checkin_week_no,checkin_day_of_year,checkin_quarter,checkin_week_of_yr,child_adult_ratio,num_single_rm_nts_adults,num_single_rm_nts_children,adult_Frac,child_Frac,pax_rm_nts,is_children,cont1,delta_rmnts_checin_in_out_delta,calc_pax_rm_nts,visit_number,last_trip_diff,next_trip_diff,last_total_pax_diff,last_roomnights_diff,last_adult_diff,last_trip_diff_check_in,last_trip_diff_check_out,next_trip_diff_check_in,next_trip_diff_check_out,last_cacl_rmnts_diff_check_in,pax_cum_mean,numberofadults_cum_mean,roomnights_cum_mean,checkin_out_delta_cum_mean,booking_checkin_delta_cum_mean,booking_checkout_delta_cum_mean,visit_number_resort,last_total_pax_diff_resort,last_roomnights_diff_resort,last_adult_diff_resort,pax_cum_mean_resort,numberofadults_cum_mean_resort,roomnights_cum_mean_resort,checkin_out_delta_cum_mean_resort,booking_checkin_delta_cum_mean_resort,booking_checkout_delta_cum_mean_resort,visit_number_state_ind,last_total_pax_diff_state_ind,last_roomnights_diff_state_ind,last_adult_diff_state_ind,pax_cum_mean_state_ind,numberofadults_cum_mean_state_ind,roomnights_cum_mean_state_ind,checkin_out_delta_cum_mean_state_ind,booking_checkin_delta_cum_mean_state_ind,booking_checkout_delta_cum_mean_state_ind,visit_number_state_res,last_total_pax_diff_state_res,last_roomnights_diff_state_res,last_adult_diff_state_res,pax_cum_mean_state_res,numberofadults_cum_mean_state_res,roomnights_cum_mean_state_res,checkin_out_delta_cum_mean_state_res,booking_checkin_delta_cum_mean_state_res,booking_checkout_delta_cum_mean_state_res
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2018-04-05,2018-04-05,2018-04-06,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,1,0,1,4,5,2018,3,14,95,2,14,4,5,2018,3,14,95,2,14,0.000000,2,0,0.666667,0.0,3,0,0.000000,0,3,14.0,67.0,-1.0,0.0,-2.0,0.0,-1.0,-2.0,-1.0,-1.0,-1.0,2.307692,2.0,3.538462,3.153846,43.230769,46.384615,4618.0,-1,-2,-1,3.427426,3.233536,3.535095,2.585572,49.105936,51.691508,29983.5,-1,-2,0,3.144739,3.469878,3.880679,2.428281,49.639436,52.067716,38688.5,1,-1,0,3.141313,3.131851,3.854757,2.512229,51.311169,53.823397
1,03930f033646d073462b35d411616323597715ac4fc398...,2015-01-23,2015-04-11,2015-04-16,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,5,78,83,1,23,2015,4,4,23,1,4,4,11,2015,5,15,101,2,15,0.000000,10,0,1.000000,0.0,10,0,0.939759,0,10,1.0,-1.0,-5.0,-1.0,-1.0,-1.0,-1.0,-1.0,69.0,70.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,238.5,-1,-16,-12,2.794118,3.331933,4.831933,2.647059,61.718487,64.365546,1158.5,0,3,0,2.871528,3.351562,4.097222,2.684896,65.642361,68.327257,1665.0,0,2,0,3.155141,3.470836,4.150932,2.697535,58.927240,61.624775
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,2015-01-28,2015-02-01,2015-02-05,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,4,4,8,1,28,2015,2,5,28,1,5,2,1,2015,6,5,32,1,5,0.000000,8,0,1.000000,0.0,8,0,0.500000,0,8,2.0,5.0,-94.0,0.0,-1.0,0.0,-69.0,-70.0,-130.0,-131.0,-1.0,2.000000,2.0,5.000000,5.000000,78.000000,83.000000,879.5,0,1,0,3.078231,3.129252,3.591837,2.547619,80.182540,82.730159,1227.0,-2,2,-2,2.864046,3.346437,4.082719,2.676495,64.180180,66.856675,4616.0,-2,0,-1,2.99

In [30]:
df['resort_id_booking_date_unique']=df.resort_id.map(df.groupby('resort_id')['booking_date'].nunique().to_dict())
df['resort_id_checkin_date_unique']=df.resort_id.map(df.groupby('resort_id')['checkin_date'].nunique().to_dict())
df['resort_id_product_code_unique']=df.resort_id.map(df.groupby('resort_id')['main_product_code'].nunique().to_dict())
df['resort_id_persontravellingid_unique']=df.resort_id.map(df.groupby('resort_id')['persontravellingid'].nunique().to_dict())
df['resort_id_member_id_unique']=df.resort_id.map(df.groupby('resort_id')['memberid'].nunique().to_dict())
df['resorts_in_a_state']=df.state_code_resort.map(df.groupby('state_code_resort')['resort_id'].nunique().to_dict())
df['resorts_in_a_region']=df.resort_region_code.map(df.groupby('resort_region_code')['resort_id'].nunique().to_dict())
df['resorts_in_a_cluster']=df.cluster_code.map(df.groupby('cluster_code')['resort_id'].nunique().to_dict())
df['resorts_in_a_resort_type']=df.resort_type_code.map(df.groupby('resort_type_code')['resort_id'].nunique().to_dict())

df['person_state_nunique']=df.state_code_residence.map(df.groupby('state_code_residence')['memberid'].nunique().to_dict())

df['resort_id_product_code_unique']=df.resort_id.map(df.groupby('resort_id')['main_product_code'].nunique().to_dict())
df['resort_id_persontravellingid_unique']=df.resort_id.map(df.groupby('resort_id')['persontravellingid'].nunique().to_dict())



In [31]:
df.shape

(488189, 109)

In [32]:
df.to_csv('temp_df.csv',index=False)

#### loading pre saved data till this point

In [34]:
df['cat_1']= df['resort_id'].astype('str')+'_'+df['memberid'].astype('str')
df['cat_2']= df['resort_id'].astype('str')+'_'+df['state_code_residence'].astype('str')
df['cat_3']= df['resort_id'].astype('str')+'_'+df['state_code_resort'].astype('str')
df['cat_4']= df['resort_id'].astype('str')+'_'+df['checkin_date'].astype('str')
df['cat_5']= df['resort_id'].astype('str')+'_'+df['checkout_date'].astype('str')
df['cat_6']= df['state_code_residence'].astype('str')+'_'+df['state_code_residence'].astype('str')
df['cat_7']= df['memberid'].astype('str')+'_'+df['checkin_date'].astype('str')
df['cat_8']= df['memberid'].astype('str')+'_'+df['checkout_date'].astype('str')
df['cat_9']= df['memberid'].astype('str')+'_'+df['cluster_code'].astype('str')
df['cat_10']= df['memberid'].astype('str')+'_'+df['booking_date'].astype('str')

In [35]:
cat_cols=['booking_date', 'checkin_date', 'checkout_date', 'channel_code', 
          'main_product_code', 'persontravellingid',
          'resort_region_code', 'resort_type_code', 'room_type_booked_code', 
          'season_holidayed_code', 'state_code_residence', 'state_code_resort', 
          'member_age_buckets', 'booking_type_code', 'memberid', 'cluster_code', 'reservationstatusid_code',
          'resort_id', 'booking_mon','booking_day', 'booking_year', 'booking_dow', 
          'booking_week_no', 'booking_day_of_year', 'booking_quarter', 
          'booking_week_of_yr', 'checkin_mon', 'checkin_day', 'checkin_year', 'checkin_dow',
          'checkin_week_no', 'checkin_day_of_year', 'checkin_quarter', 'checkin_week_of_yr',
         'cat_1','cat_2','cat_3','cat_4','cat_5','cat_6',
          'cat_7','cat_8','cat_9','cat_10',
         ]

In [36]:
def level_1_group_by_feats(df,cat,num):
    df[cat+'_'+num+'_mean']=df[cat].map(dict(df.groupby([cat])[num].mean()))
    df[cat+'_'+num+'_std']=df[cat].map(dict(df.groupby([cat])[num].std()))
    df[cat+'_'+num+'_mean_delta']=df[num]- df[cat+'_'+num+'_mean']
    df[cat+'_'+num+'_mean_ratio']=df[num]/ (1+df[cat+'_'+num+'_mean'])
    return df

In [37]:
num_cols=['numberofadults', 'numberofchildren','roomnights','total_pax','checkin_out_delta','booking_checkin_delta',
         'booking_checkout_delta','child_adult_ratio','num_single_rm_nts_adults','num_single_rm_nts_children',
         'adult_Frac','child_Frac','pax_rm_nts','is_children','cont1','delta_rmnts_checin_in_out_delta',
         'calc_pax_rm_nts',
         'pax_cum_mean_state_ind', 'checkin_out_delta_cum_mean_resort','visit_number_resort',
          'visit_number_state_ind','visit_number_state_res','pax_cum_mean_resort']

In [38]:
print len(cat_cols), len(num_cols)

44 23


In [ ]:
%%time
cats=['state_code_resort','state_code_residence','memberid','resort_id','checkin_date', 'checkout_date','cat_2','cat_7','cat_8']
combos=[(a,b) for a in num_cols for b in cats]
print len(combos)
for i,combo in enumerate(combos):
    df=level_1_group_by_feats(df,combo[1],combo[0])
    print i

207
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203


In [48]:
df.shape

In [ ]:
fe=FreqeuncyEncoding(categorical_columns=cat_cols,normalize=False,return_df=True)
df=fe.fit_transform(df)

In [ ]:
df.head()

In [ ]:
train=df[df.reservation_id.isin(train.reservation_id)]
test=df[df.reservation_id.isin(test.reservation_id)]
train.drop('reservation_id',axis=1,inplace=True)
test.drop('reservation_id',axis=1,inplace=True)
print train.shape, test.shape

In [ ]:
del df

In [ ]:
import gc
gc.collect()

In [49]:
print train.shape, test.shape

(341424, 946) (146765, 946)


In [50]:
params={'bagging_freq': 5, 'boosting_type': 'gbdt', 'colsample_bytree': 0.4000000000000001,
 'learning_rate': 0.01, 'min_child_weight': 300.0, 'n_estimators': 15000,
 'n_jobs': -1, 'num_leaves': 64,'subsample': 1.0}

In [51]:
mod=Estimator(model=LGBMRegressor(**params),n_jobs=-1,
                          validation_scheme='KFold',
              random_state=100,n_splits=5,early_stopping_rounds=200,shuffle=True)

In [52]:
%%time
oof=mod.fit_transform(train.values,target.values)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's rmse: 1.00462	valid_0's l2: 1.00927	valid_1's rmse: 1.00205	valid_1's l2: 1.00411
[200]	valid_0's rmse: 0.982494	valid_0's l2: 0.965295	valid_1's rmse: 0.978048	valid_1's l2: 0.956579
[300]	valid_0's rmse: 0.973389	valid_0's l2: 0.947486	valid_1's rmse: 0.96653	valid_1's l2: 0.934181
[400]	valid_0's rmse: 0.968955	valid_0's l2: 0.938875	valid_1's rmse: 0.959438	valid_1's l2: 0.920521
[500]	valid_0's rmse: 0.966541	valid_0's l2: 0.934202	valid_1's rmse: 0.954163	valid_1's l2: 0.910426
[600]	valid_0's rmse: 0.965078	valid_0's l2: 0.931375	valid_1's rmse: 0.949812	valid_1's l2: 0.902142
[700]	valid_0's rmse: 0.964157	valid_0's l2: 0.929599	valid_1's rmse: 0.945964	valid_1's l2: 0.894848
[800]	valid_0's rmse: 0.963451	valid_0's l2: 0.928238	valid_1's rmse: 0.942385	valid_1's l2: 0.888089
[900]	valid_0's rmse: 0.962974	valid_0's l2: 0.927319	valid_1's rmse: 0.939044	valid_1's l2: 0.881803
[1000]	valid_0's rmse: 0

[3900]	valid_0's rmse: 0.961363	valid_0's l2: 0.924219	valid_1's rmse: 0.866581	valid_1's l2: 0.750962
[4000]	valid_0's rmse: 0.961367	valid_0's l2: 0.924226	valid_1's rmse: 0.864467	valid_1's l2: 0.747304
[4100]	valid_0's rmse: 0.961329	valid_0's l2: 0.924153	valid_1's rmse: 0.862372	valid_1's l2: 0.743686
[4200]	valid_0's rmse: 0.961301	valid_0's l2: 0.924099	valid_1's rmse: 0.860296	valid_1's l2: 0.740109
[4300]	valid_0's rmse: 0.961298	valid_0's l2: 0.924094	valid_1's rmse: 0.858236	valid_1's l2: 0.73657
[4400]	valid_0's rmse: 0.96127	valid_0's l2: 0.924039	valid_1's rmse: 0.856158	valid_1's l2: 0.733007
[4500]	valid_0's rmse: 0.96127	valid_0's l2: 0.924041	valid_1's rmse: 0.854143	valid_1's l2: 0.729561
[4600]	valid_0's rmse: 0.961274	valid_0's l2: 0.924047	valid_1's rmse: 0.852096	valid_1's l2: 0.726067
[4700]	valid_0's rmse: 0.961286	valid_0's l2: 0.924071	valid_1's rmse: 0.850115	valid_1's l2: 0.722695
[4800]	valid_0's rmse: 0.961253	valid_0's l2: 0.924007	valid_1's rmse: 0.848

[2100]	valid_0's rmse: 0.970171	valid_0's l2: 0.941231	valid_1's rmse: 0.904834	valid_1's l2: 0.818724
[2200]	valid_0's rmse: 0.970126	valid_0's l2: 0.941145	valid_1's rmse: 0.902444	valid_1's l2: 0.814405
[2300]	valid_0's rmse: 0.970035	valid_0's l2: 0.940967	valid_1's rmse: 0.900126	valid_1's l2: 0.810227
[2400]	valid_0's rmse: 0.969976	valid_0's l2: 0.940853	valid_1's rmse: 0.897791	valid_1's l2: 0.806028
[2500]	valid_0's rmse: 0.969928	valid_0's l2: 0.940759	valid_1's rmse: 0.89549	valid_1's l2: 0.801902
[2600]	valid_0's rmse: 0.969898	valid_0's l2: 0.940703	valid_1's rmse: 0.893234	valid_1's l2: 0.797866
[2700]	valid_0's rmse: 0.96985	valid_0's l2: 0.940609	valid_1's rmse: 0.890959	valid_1's l2: 0.793809
[2800]	valid_0's rmse: 0.969798	valid_0's l2: 0.940507	valid_1's rmse: 0.888726	valid_1's l2: 0.789833
[2900]	valid_0's rmse: 0.969747	valid_0's l2: 0.94041	valid_1's rmse: 0.886483	valid_1's l2: 0.785852
[3000]	valid_0's rmse: 0.96976	valid_0's l2: 0.940435	valid_1's rmse: 0.8842

In [53]:
print mod.cv_scores, mod.avg_cv_score, np.std(mod.cv_scores)

[0.9600251996730352, 0.9611776768179838, 0.9512363455456423, 0.9694273721653577, 0.9664131242519836] 0.9616559436908005 0.006236697173769968


In [53]:
mod2=Estimator(model=LGBMRegressor(**params),n_jobs=-1,
                          validation_scheme='KFold',
              random_state=200,n_splits=5,early_stopping_rounds=200,shuffle=True)

In [55]:
%%time
oof2=mod2.fit_transform(train.values,target.values)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's rmse: 1.00261	valid_0's l2: 1.00522	valid_1's rmse: 1.00214	valid_1's l2: 1.00428
[200]	valid_0's rmse: 0.981371	valid_0's l2: 0.963088	valid_1's rmse: 0.97811	valid_1's l2: 0.9567
[300]	valid_0's rmse: 0.972815	valid_0's l2: 0.94637	valid_1's rmse: 0.966529	valid_1's l2: 0.934178
[400]	valid_0's rmse: 0.968747	valid_0's l2: 0.93847	valid_1's rmse: 0.959407	valid_1's l2: 0.920461
[500]	valid_0's rmse: 0.966481	valid_0's l2: 0.934085	valid_1's rmse: 0.954136	valid_1's l2: 0.910375
[600]	valid_0's rmse: 0.965169	valid_0's l2: 0.931551	valid_1's rmse: 0.949764	valid_1's l2: 0.902051
[700]	valid_0's rmse: 0.964315	valid_0's l2: 0.929904	valid_1's rmse: 0.945923	valid_1's l2: 0.89477
[800]	valid_0's rmse: 0.963674	valid_0's l2: 0.928667	valid_1's rmse: 0.942324	valid_1's l2: 0.887974
[900]	valid_0's rmse: 0.963212	valid_0's l2: 0.927777	valid_1's rmse: 0.938961	valid_1's l2: 0.881648
[1000]	valid_0's rmse: 0.9628

[4200]	valid_0's rmse: 0.957769	valid_0's l2: 0.917321	valid_1's rmse: 0.861264	valid_1's l2: 0.741776
[4300]	valid_0's rmse: 0.957758	valid_0's l2: 0.917301	valid_1's rmse: 0.859179	valid_1's l2: 0.738189
[4400]	valid_0's rmse: 0.957753	valid_0's l2: 0.91729	valid_1's rmse: 0.857143	valid_1's l2: 0.734695
[4500]	valid_0's rmse: 0.957759	valid_0's l2: 0.917302	valid_1's rmse: 0.855091	valid_1's l2: 0.73118
[4600]	valid_0's rmse: 0.957695	valid_0's l2: 0.917179	valid_1's rmse: 0.853001	valid_1's l2: 0.727611
[4700]	valid_0's rmse: 0.957681	valid_0's l2: 0.917152	valid_1's rmse: 0.850946	valid_1's l2: 0.724109
[4800]	valid_0's rmse: 0.957631	valid_0's l2: 0.917057	valid_1's rmse: 0.848878	valid_1's l2: 0.720594
[4900]	valid_0's rmse: 0.957639	valid_0's l2: 0.917073	valid_1's rmse: 0.84679	valid_1's l2: 0.717054
[5000]	valid_0's rmse: 0.957653	valid_0's l2: 0.9171	valid_1's rmse: 0.844804	valid_1's l2: 0.713695
Early stopping, best iteration is:
[4860]	valid_0's rmse: 0.957621	valid_0's l

[3300]	valid_0's rmse: 0.961732	valid_0's l2: 0.924929	valid_1's rmse: 0.879143	valid_1's l2: 0.772893
[3400]	valid_0's rmse: 0.961711	valid_0's l2: 0.924888	valid_1's rmse: 0.876975	valid_1's l2: 0.769085
[3500]	valid_0's rmse: 0.961696	valid_0's l2: 0.924858	valid_1's rmse: 0.874798	valid_1's l2: 0.765272
[3600]	valid_0's rmse: 0.961687	valid_0's l2: 0.924842	valid_1's rmse: 0.872652	valid_1's l2: 0.761521
[3700]	valid_0's rmse: 0.961685	valid_0's l2: 0.924838	valid_1's rmse: 0.870534	valid_1's l2: 0.75783
Early stopping, best iteration is:
[3568]	valid_0's rmse: 0.961659	valid_0's l2: 0.924788	valid_1's rmse: 0.873314	valid_1's l2: 0.762677
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's rmse: 1.00896	valid_0's l2: 1.01799	valid_1's rmse: 1.00078	valid_1's l2: 1.00156
[200]	valid_0's rmse: 0.988021	valid_0's l2: 0.976185	valid_1's rmse: 0.976639	valid_1's l2: 0.953824
[300]	valid_0's rmse: 0.979211	valid_0's l2: 0.958854	valid_1's rmse: 0.965073	valid_

In [56]:
print mod2.cv_scores, mod2.avg_cv_score, np.std(mod2.cv_scores)

[0.9609463240747629, 0.9576207615464972, 0.9609933089097317, 0.9616589514534485, 0.9657418199933169] 0.9613922331955514 0.002591216620424853


In [57]:
preds1=mod.predict(test.values)
preds2=mod2.predict(test.values)

custom_estimator_regressor.py:108: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict(x) for est in self.fitted_models)), axis=1)


In [58]:
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':preds1})
sol.to_csv('preds_lgb_946_seed_100.csv',index=False)

In [59]:
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':preds2})
sol.to_csv('preds_lgb_946_seed_200.csv',index=False)

In [60]:
res_ids=pd.read_csv('train.csv',usecols=['reservation_id']).reservation_id
oof_df=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':oof2})
oof_df.to_csv('OOF_lgb_946_seed_200.csv',index=False)

In [61]:
res_ids=pd.read_csv('train.csv',usecols=['reservation_id']).reservation_id
oof_df=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':oof})
oof_df.to_csv('OOF_lgb_946_seed_100.csv',index=False)

In [63]:
# print np.corrcoef(oof,pd.read_csv('../av_club_mahindra/OOF_lgb_946_seed_100.csv').iloc[:,0])
# print np.corrcoef(preds1,pd.read_csv('../av_club_mahindra/preds_lgb_946_seed_100.csv').iloc[:,0])

In [68]:
# print np.corrcoef(oof2,pd.read_csv('../av_club_mahindra/OOF_lgb_946_seed_200.csv').iloc[:,0])
# print np.corrcoef(preds2,pd.read_csv('../av_club_mahindra/preds_lgb_946_seed_200.csv').iloc[:,0])